In [328]:
import pandas as pd

In [329]:
LOCAL_PARQUET_PATH = 'storage/todo.parquet'
CHUNK_SIZE = 200
df = pd.read_parquet(LOCAL_PARQUET_PATH)

In [ ]:
def statusUpdate(df):
    total_chunks = len(df) // CHUNK_SIZE + (1 if len(df) % CHUNK_SIZE != 0 else 0)
    processed_chunks = 0
    
    non_null_rows = 0
    last_five_non_null = [0] * 5 # Track non-null counts for the last five chunks
    
    for i in range(0, len(df), CHUNK_SIZE):
        chunk = df.iloc[i:i+CHUNK_SIZE]

        non_null_count = chunk['rawText'].notnull().sum()
        non_null_rows += non_null_count
        last_five_non_null[processed_chunks % 5] = non_null_count

        # Check if the entire chunk's rawText column is null
        if chunk['rawText'].isnull().all():
            processed_percentage = (processed_chunks / total_chunks) * 100
            conversion_rate = (non_null_rows / len(df)) * 100
            last_five_conversion_rate = (sum(last_five_non_null) / (CHUNK_SIZE * 5)) * 100
            
            # Print results in a more readable format
            print(f"\n{'Progress Summary':^40}")
            print(f"{'Processed Chunks':<20}{'Total Chunks':<20}{'Percentage':<20}")
            print(f"{processed_chunks:<20}{total_chunks:<20}{processed_percentage:.2f}%")
            
            print("\nConversion Rates:")
            print(f"{'Metric':<30}{'Value'}")
            print(f"{'Overall Conversion Rate':<30}{conversion_rate:.2f}%")
            print(f"{'Last 5 Chunks Conversion Rate':<30}{last_five_conversion_rate:.2f}%")

            preview = pd.concat([df.iloc[max(0, i-CHUNK_SIZE):i].tail(5), df.iloc[i:i+CHUNK_SIZE].head(5)])
            return preview

        processed_chunks += 1
preview = statusUpdate(df)
preview